# Leer registros de pruebas

In [48]:
import pandas as pd

df = pd.read_csv("experimentos_log.csv")

# Vista rápida
print(df.head())

# Ver cuántos experimentos guardaste
print("Total experimentos:", len(df))

# Ver qué algoritmos probaste
print(df["algoritmo"].unique())

             timestamp                                  script  \
0  2025-09-04T17:39:51   20250901_PruebasEntrenamientoRF.ipynb   
1  2025-09-04T17:59:07  20250901_PruebasEntrenamientoDNN.ipynb   
2  2025-09-04T18:25:25  20250901_PruebasEntrenamientoViT.ipynb   
3  2025-09-04T18:25:57  20250901_PruebasEntrenamientoXGB.ipynb   

               algoritmo                                  dataset  \
0           RandomForest                PuntosMuestra_CR_2023.csv   
1            MLP_tabular                PuntosMuestra_CR_2023.csv   
2  ViT_tiny_linear_probe  PuntosMuestra_CR_tinto_synthetic_images   
3                XGBoost                PuntosMuestra_CR_2023.csv   

   clases_removidas  seed  n_train  n_test  n_features  num_classes  \
0                 2    42    20439    8760        13.0            6   
1                 2    42     8760    8760         NaN            6   
2                 2    42    20439    8760         NaN            6   
3                 2    42    20439    8

In [49]:
with open("experimentos_log.csv", encoding="utf-8") as f:
    for i in range(5):
        print(f.readline())


timestamp,script,algoritmo,dataset,clases_removidas,seed,n_train,n_test,n_features,num_classes,fit_seconds,pred_seconds,ms_per_sample,OA,F1_macro,system_info

2025-09-04T17:39:51,20250901_PruebasEntrenamientoRF.ipynb,RandomForest,PuntosMuestra_CR_2023.csv,2,42,20439,8760,13.0,6,1.0931,0.0719,0.008,0.8826,0.8161,"{""host"": ""IALB18"", ""user"": ""Fran"", ""os"": ""Windows 10"", ""machine"": ""AMD64"", ""processor"": ""Intel64 Family 6 Model 183 Stepping 1, GenuineIntel"", ""python"": ""3.11.13"", ""cpu_count_logical"": 32, ""cpu_count_physical"": 24, ""ram_gb"": 127.71, ""numpy"": ""2.3.2"", ""pandas"": ""2.3.2"", ""sklearn"": ""1.7.1"", ""torch"": ""2.8.0+cpu"", ""torchvision"": ""0.23.0+cpu"", ""timm"": ""1.0.19"", ""xgboost"": ""3.0.4"", ""cuda_available"": false, ""cuda_version"": null, ""gpu_name"": """"}"

2025-09-04T17:59:07,20250901_PruebasEntrenamientoDNN.ipynb,MLP_tabular,PuntosMuestra_CR_2023.csv,2,42,8760,8760,,6,287.6801,30.0637,3.432,0.5234,0.1145,"{""host"": ""IALB18"", 